In [48]:
import base64
import datetime
import json
import os
import numpy as np
import pandas as pd

In [49]:
# Read the data file and take a look at the data
df = pd.read_csv('cleaneddata.txt', sep=",")
df.drop([0], axis=0, inplace=True)
df.drop(['Filler'], axis=1 , inplace = True) #it is all blank information
df.drop(['School Code'], axis=1, inplace = True) #School code is 0000000
df.drop(['Area 1 Percentage Above Standard',
       'Area 1 Percentage Near Standard', 'Area 1 Percentage Below Standard',
       'Area 2 Percentage Above Standard', 'Area 2 Percentage Near Standard',
       'Area 2 Percentage Below Standard', 'Area 3 Percentage Above Standard',
       'Area 3 Percentage Near Standard', 'Area 3 Percentage Below Standard',
       'Area 4 Percentage Above Standard', 'Area 4 Percentage Near Standard',
       'Area 4 Percentage Below Standard'], axis=1, inplace=True)
df.drop(['Test Year','Test Type'], axis=1, inplace=True) #drop because we know is only 2017, test type is B for all
df=df.reset_index()
df.head()
df.drop(['index'], axis = 1, inplace=True)
df.head()
df=df.apply(pd.to_numeric, errors='coerce')

In [50]:
df.head(20) 
#We can see that we have several grades, several test Id and also several subgroups. We will require to group by some of this columns in order to gather the data for the distsrict.
#To aggregate the outcomes percentages we will convert them to qty first to avoid percentages over 100%

,County Code,District Code,Subgroup ID,Total Tested At Entity Level,Total Tested with Scores,Grade,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores
0,1,10017,1,51.0,51.0,6,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10017,1,44.0,44.0,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,10017,1,51.0,51.0,7,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,10017,1,44.0,44.0,7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10017,1,44.0,44.0,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,10017,1,51.0,51.0,8,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,10017,1,51.0,51.0,11,2,151.0,44.0,2412.8,0.0,0.00,0.00,4.55,95.45,44.0
7,1,10017,1,44.0,44.0,11,1,150.0,38.0,2435.4,0.0,5.26,5.26,21.05,73.68,38.0
8,1,10017,1,51.0,51.0,13,2,177.0,51.0,NaN,0.0,0.00,0.00,3.92,96.08,51.0
9,1,10017,1,44.0,44.0,13,1,175.0,44.0,NaN,0.0,4.55,4.55,18.18,77.27,44.0


In [51]:
df.columns

Index(['County Code', 'District Code', 'Subgroup ID',
       'Total Tested At Entity Level', 'Total Tested with Scores', 'Grade',
       'Test Id', 'CAASPP Reported Enrollment', 'Students Tested',
       'Mean Scale Score', 'Percentage Standard Exceeded',
       'Percentage Standard Met', 'Percentage Standard Met and Above',
       'Percentage Standard Nearly Met', 'Percentage Standard Not Met',
       'Students with Scores'],
      dtype='object')

In [5]:
df.Grade.unique()

array([ 6,  7,  8, 11, 13,  4,  5,  3])

In [6]:
#Will drop Grade 13 to see if our numbers align properly
#df.loc[df['Language'].apply(lambda x: x in ['C', 'C++'] )]
rg =[6,7,8,11,4,5,3]
df=df.loc[df['Grade'].apply(lambda x: x in rg)]

In [7]:
#To review assumptions, we will confirm the 5 column percentages are representation of 100% of the students.
#After doing this review, it is clear 'Percenage met and above has duplicate information'
#We will only consider the 4 percentage amounts now
df['Percentage Tested']=(df['Percentage Standard Exceeded'] + df['Percentage Standard Met']  + df['Percentage Standard Nearly Met'] + df['Percentage Standard Not Met']).round(decimals=0)

In [8]:
dfp=df['Percentage Tested']
dfp=dfp.dropna()
dfp

6         100.0
7         100.0
16        100.0
17        100.0
24        100.0
25        100.0
34        100.0
35        100.0
54        100.0
55        100.0
64        100.0
65        100.0
86        100.0
87        100.0
112       100.0
113       100.0
142       100.0
143       100.0
150       100.0
151       100.0
170       100.0
171       100.0
207       100.0
208       100.0
217       100.0
218       100.0
517       100.0
518       100.0
519       100.0
520       100.0
          ...  
474858    100.0
474859    100.0
474874    100.0
474875    100.0
474880    100.0
474881    100.0
474890    100.0
474891    100.0
474894    100.0
474895    100.0
474898    100.0
474899    100.0
474902    100.0
474903    100.0
474908    100.0
474909    100.0
474914    100.0
474915    100.0
474938    100.0
474939    100.0
474944    100.0
474945    100.0
474956    100.0
474957    100.0
474960    100.0
474961    100.0
474976    100.0
474977    100.0
474982    100.0
474983    100.0
Name: Percentage Tested,

In [9]:
count=0
for i in dfp:
    if i!=100:
        count=+1
print(count)

0


#After doing this review, it is clear 'Percenage met and above has duplicate information', so we removed it from the calculation of total percentages of students tested.
We will keep the Column for information only.

In [10]:
#Converting all the assessment columns from percentage columns to absolute numbers for exceeded, nearly met, not met
df['Total Standard Exceeded'] =(df['Students Tested']*df['Percentage Standard Exceeded']/100).round(decimals=0)
df['Total Standard Met'] =(df['Students Tested']*df['Percentage Standard Met']/100).round(decimals=0)
df['Total Standard Nearly Met'] =(df['Students Tested']*df['Percentage Standard Nearly Met']/100).round(decimals=0)
df['Total Standard Not Met'] =(df['Students Tested']*df['Percentage Standard Not Met']/100).round(decimals=0)
df['Total Standard Met and Above'] =(df['Students Tested']*df['Percentage Standard Met and Above']/100).round(decimals=0)
#Percentage was multiplied by the students tested information and not the total enrollment information.
#Students tested in Math and Literacy can be the same students or different students, we don't have that information.
#we are adding the results of both exams, this can cause a duplicate in value of students tested, I will separate both datasets

In [11]:
df.head(10)

,County Code,District Code,Subgroup ID,Total Tested At Entity Level,Total Tested with Scores,Grade,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,...,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores,Percentage Tested,Total Standard Exceeded,Total Standard Met,Total Standard Nearly Met,Total Standard Not Met,Total Standard Met and Above
0,1,10017,1,51.0,51.0,6,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10017,1,44.0,44.0,6,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,10017,1,51.0,51.0,7,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,10017,1,44.0,44.0,7,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10017,1,44.0,44.0,8,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,10017,1,51.0,51.0,8,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,10017,1,51.0,51.0,11,2,151.0,44.0,2412.8,...,0.00,4.55,95.45,44.0,100.0,0.0,0.0,2.0,42.0,0.0
7,1,10017,1,44.0,44.0,11,1,150.0,38.0,2435.4,...,5.26,21.05,73.68,38.0,100.0,0.0,2.0,8.0,28.0,2.0
10,1,10017,3,30.0,30.0,6,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,1,10017,3,36.0,36.0,6,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#We will require to create two assessment datasets, one with the total number of students (subgroup id 1) and another with the ethnicity (Subgroup 74.75.76.77.78.79.80) The subgroup 144 
#has two - races. I will bring it over and confirm the addition of subgroup 1 is equal to the ethnicity groups addition.

In [13]:
df_subgroups = pd.read_csv('Subgroups.txt', sep=",")
df_subgroups

,id,subgroup_id,subgroup_definition,subgroup_type
0,1,1,"""All Students""","""All Students"""
1,3,3,"""Males""","""Gender"""
2,4,4,"""Females""","""Gender"""
3,6,6,"""Fluent-English Proficient and English Only""","""English-Language Fluency"""
4,7,7,"""Initially-Fluent English Proficient (IFEP)""","""English-Language Fluency"""
5,8,8,"""Reclassified-Fluent English Proficient (RFEP)""","""English-Language Fluency"""
6,28,28,"""Education""","""Migrant"""
7,31,31,"""EconomicallMigranty Disadvantaged""","""Economic Status"""
8,74,74,"""Black or African American""","""Ethnicity"""
9,75,75,"""American Indian or Alaska Native""","""Ethnicity"""


1   'ALL STUDENTS"
74	"Black or African American"	"Ethnicity"
75	75	"American Indian or Alaska Native"	"Ethnicity"
76	76	"Asian"	"Ethnicity"
77	77	"Filipino"	"Ethnicity"
78	78	"Hispanic or Latino"	"Ethnicity"
79	79	"Native Hawaiian or Pacific Islander"	"Ethnicity"
80	80	"White"	"Ethnicity"
144     "Two or more races"

200- 2007 - ethnicity for economically disadvantaged
220-227 - ethnicity for not economically disadvantaged

In [16]:
subgroups_sg= [1,74,75,76,77,78,79,80,144]
#df.loc[df['Language'].apply(lambda x: x in ['C', 'C++'] )]
subgroups_selected = df.loc[df['Subgroup ID'].apply(lambda x: x in subgroups_sg)]
subgroups_selected.head(5)

,County Code,District Code,Subgroup ID,Total Tested At Entity Level,Total Tested with Scores,Grade,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,...,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores,Total Standard Exceeded,Total Standard Met,Total Standard Nearly Met,Total Standard Not Met,Total Standard Met and Above,Percentage Tested
0,1,10017,1,51.0,51.0,6,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10017,1,44.0,44.0,6,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,10017,1,51.0,51.0,7,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,10017,1,44.0,44.0,7,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10017,1,44.0,44.0,8,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
eco_subgroups_sg= [200,201,202,203,204,205,206,207,220,221,222,223,224,225,226,227]
#df.loc[df['Language'].apply(lambda x: x in ['C', 'C++'] )]
eco_subgroups_selected = df.loc[df['Subgroup ID'].apply(lambda x: x in eco_subgroups_sg)]
eco_subgroups_selected.head(5)

,County Code,District Code,Subgroup ID,Total Tested At Entity Level,Total Tested with Scores,Grade,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,...,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores,Percentage Tested,Total Standard Exceeded,Total Standard Met,Total Standard Nearly Met,Total Standard Not Met,Total Standard Met and Above
211,1,10017,200,17.0,17.0,6,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212,1,10017,200,18.0,18.0,6,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213,1,10017,200,17.0,17.0,7,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214,1,10017,200,18.0,18.0,7,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215,1,10017,200,18.0,18.0,8,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
subgroups_selected = subgroups_selected.drop(columns=['Percentage Standard Exceeded','Percentage Standard Met and Above', 'Percentage Standard Met','Percentage Standard Met and Above','Percentage Standard Nearly Met','Percentage Standard Not Met'])

NameError: name 'subgroups_selected' is not defined

In [37]:
eco_subgroups_selected = eco_subgroups_selected.drop(columns=['Percentage Standard Exceeded','Percentage Standard Met and Above', 'Percentage Standard Met','Percentage Standard Met and Above','Percentage Standard Nearly Met','Percentage Standard Not Met'])

In [38]:
subgroups_selected.columns

NameError: name 'subgroups_selected' is not defined

In [39]:
eco_subgroups_selected.columns

Index(['County Code', 'District Code', 'Subgroup ID',
       'Total Tested At Entity Level', 'Total Tested with Scores', 'Grade',
       'Test Id', 'CAASPP Reported Enrollment', 'Students Tested',
       'Mean Scale Score', 'Students with Scores', 'Percentage Tested',
       'Total Standard Exceeded', 'Total Standard Met',
       'Total Standard Nearly Met', 'Total Standard Not Met',
       'Total Standard Met and Above'],
      dtype='object')

In [19]:
df_mean=subgroups_selected.groupby(['County Code','District Code','Subgroup ID','Test Id']).agg({'Mean Scale Score':'mean'}).reset_index()

In [40]:
df_eco_mean=eco_subgroups_selected.groupby(['County Code','District Code','Subgroup ID','Test Id']).agg({'Mean Scale Score':'mean'}).reset_index()

In [31]:
df_ethnicity_new=subgroups_selected.groupby(['County Code','District Code','Subgroup ID','Test Id'])['Total Tested At Entity Level', 'Total Tested with Scores', 'CAASPP Reported Enrollment', 'Students Tested'
       ,'Students with Scores','Total Standard Exceeded', 'Total Standard Met',
       'Total Standard Met and Above', 'Total Standard Nearly Met',
       'Total Standard Not Met'].sum()
df_ethnicity_new=df_ethnicity_new.reset_index()

NameError: name 'subgroups_selected' is not defined

In [41]:
df_eco_ethnicity_new=eco_subgroups_selected.groupby(['County Code','District Code','Subgroup ID','Test Id'])['Total Tested At Entity Level', 'Total Tested with Scores', 'CAASPP Reported Enrollment', 'Students Tested'
       ,'Students with Scores','Total Standard Exceeded', 'Total Standard Met',
       'Total Standard Met and Above', 'Total Standard Nearly Met',
       'Total Standard Not Met'].sum()
df_eco_ethnicity_new=df_eco_ethnicity_new.reset_index()

In [42]:
new_df=pd.merge(df_mean, df_ethnicity_new, how='outer', on=['County Code','District Code','Subgroup ID','Test Id'])
new_df= new_df.sort_values(by=['District Code'])
new_df=new_df.dropna()

NameError: name 'df_mean' is not defined

In [43]:
new_eco_df=pd.merge(df_eco_mean, df_eco_ethnicity_new, how='outer', on=['County Code','District Code','Subgroup ID','Test Id'])
new_eco_df= new_eco_df.sort_values(by=['District Code'])
new_eco_df=new_eco_df.dropna()

In [44]:
new_df.head(20)
#This table contains all of the grades together separated by all students and by ethnicities as well as type of test.

NameError: name 'new_df' is not defined

In [45]:
new_eco_df.describe()

,County Code,District Code,Subgroup ID,Test Id,Mean Scale Score,Total Tested At Entity Level,Total Tested with Scores,CAASPP Reported Enrollment,Students Tested,Students with Scores,Total Standard Exceeded,Total Standard Met,Total Standard Met and Above,Total Standard Nearly Met,Total Standard Not Met
count,9269.000000,9269.000000,9269.000000,9269.000000,9269.000000,9.269000e+03,9.269000e+03,9269.000000,9269.000000,9269.000000,9269.000000,9269.000000,9269.000000,9269.000000,9269.000000
mean,30.111015,66670.793182,214.127738,1.500054,2520.226949,4.045468e+03,4.042406e+03,636.087280,620.996763,620.496278,118.672133,149.781746,268.514295,153.123638,199.274571
std,14.770203,9527.429888,10.330150,0.500027,61.478301,2.026454e+04,2.024757e+04,2977.119209,2915.789492,2913.330877,391.241179,601.341941,942.201775,786.996980,1300.957855
min,1.000000,10017.000000,200.000000,1.000000,2327.700000,1.100000e+01,1.100000e+01,11.000000,11.000000,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,64642.000000,204.000000,1.000000,2474.083333,4.200000e+02,4.200000e+02,58.000000,56.000000,56.000000,7.000000,13.000000,21.000000,12.000000,11.000000
50%,33.000000,67678.000000,220.000000,2.000000,2515.583333,9.870000e+02,9.870000e+02,170.000000,165.000000,165.000000,26.000000,42.000000,70.000000,39.000000,38.000000
75%,41.000000,70615.000000,224.000000,2.000000,2560.925000,3.042000e+03,3.036000e+03,507.000000,493.000000,493.000000,92.000000,125.000000,223.000000,116.000000,114.000000
max,58.000000,77032.000000,227.000000,2.000000,2775.200000,1.216530e+06,1.215837e+06,176939.000000,173790.000000,173691.000000,15506.000000,40249.000000,55757.000000,49009.000000,85429.000000


In [24]:
#Now we will create the data set for total students
all_sg= [1]
#df.loc[df['Language'].apply(lambda x: x in ['C', 'C++'] )]
all_students_df = new_df.loc[new_df['Subgroup ID'].apply(lambda x: x in all_sg)]
all_students_df=all_students_df.dropna()
all_students_df.head(5)


,County Code,District Code,Subgroup ID,Test Id,Mean Scale Score,Total Tested At Entity Level,Total Tested with Scores,CAASPP Reported Enrollment,Students Tested,Students with Scores,Total Standard Exceeded,Total Standard Met,Total Standard Met and Above,Total Standard Nearly Met,Total Standard Not Met
0,1,10017,1,1,2435.400000,176.0,176.0,150.0,38.0,38.0,0.0,2.0,2.0,8.0,28.0
1,1,10017,1,2,2412.800000,204.0,204.0,151.0,44.0,44.0,0.0,0.0,0.0,2.0,42.0
363,3,10033,1,2,2377.840000,896.0,896.0,132.0,112.0,112.0,1.0,8.0,9.0,23.0,80.0
362,3,10033,1,1,2392.260000,917.0,917.0,132.0,115.0,115.0,2.0,18.0,20.0,20.0,75.0
390,4,10041,1,1,2451.514286,1316.0,1316.0,197.0,188.0,188.0,10.0,37.0,47.0,57.0,84.0


In [26]:
#And we will have a separate file separated by ethnicity
subgroups_sg= [74,75,76,77,78,79,80,144]
#df.loc[df['Language'].apply(lambda x: x in ['C', 'C++'] )]
ethnicity_df= new_df.loc[new_df['Subgroup ID'].apply(lambda x: x in subgroups_sg)]
ethnicity_df=ethnicity_df.dropna()
ethnicity_df.head(5)

,County Code,District Code,Subgroup ID,Test Id,Mean Scale Score,Total Tested At Entity Level,Total Tested with Scores,CAASPP Reported Enrollment,Students Tested,Students with Scores,Total Standard Exceeded,Total Standard Met,Total Standard Met and Above,Total Standard Nearly Met,Total Standard Not Met
9,1,10017,78,2,2431.30,80.0,80.0,51.0,16.0,16.0,0.0,0.0,0.0,1.0,15.0
8,1,10017,78,1,2464.90,60.0,60.0,51.0,12.0,12.0,0.0,1.0,1.0,4.0,7.0
3,1,10017,74,2,2403.70,92.0,92.0,72.0,22.0,22.0,0.0,0.0,0.0,0.0,22.0
2,1,10017,74,1,2419.60,84.0,84.0,72.0,20.0,20.0,0.0,0.0,0.0,3.0,17.0
369,3,10033,80,2,2470.05,595.0,595.0,57.0,53.0,53.0,0.0,7.0,7.0,15.0,31.0


In [27]:
all_students_df.to_csv(r'20210101/all_assessment.csv', index = False, header=True)
ethnicity_df.to_csv(r'20210101/ethnicity_new.csv', index = False, header=True)

In [46]:
new_eco_df['Subgroup ID'].unique()

array([200, 226, 206, 204, 224, 220, 222, 227, 202, 203, 207, 223, 225,
       205, 201, 221])

In [47]:
new_eco_df.to_csv(r'assessment_economic_ethnicity.csv',index=False, header=True)